In [2]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path
import timeit


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether_v2 as birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [3]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [4]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    print(cluster_structure)
    goals = ['recall','precision','pf']
    for _ in range(1):
        count = 0
        count_not = 0
        count_yes = 0
        score = []
        score_med = []
        cluster_info = {}
        for cluster in clusters:
            if cluster.rsplit('/',1)[1] in ['results','cdom_level1']:
                continue
            df = pd.read_csv(cluster + '/cdom_latest.csv')
            counts = {}
            med_count = []
            c_dom = df.wins.values.tolist()
            best_project = df.iloc[c_dom.index(max(c_dom)),0]
            for goal in goals:
                goal_df = pd.read_csv(cluster + '/1385_LR_bellwether_' + goal + '.csv')
                goal_df.rename(columns={'Unnamed: 0':'projects'},inplace=True)
                j = goal_df[goal_df['projects'] == best_project].values[0][1:]
                if goal == 'pci_20':
                    value = sum(i >= 0.40 for i in j)
                elif goal != 'pf':
                    value = sum(i >= 0.66 for i in j)
                else:
                    value = sum(i <= 0.33 for i in j)
                counts[goal] = value
            score_med.append([int(cluster.rsplit('/',1)[1]),
                              counts['recall'],
                              counts['precision'],
                              counts['pf'],
                              max(c_dom),
                              best_project])
        score_df = pd.DataFrame(score_med, columns = ['id','count_recall',
                                                      'count_precision','count_pf',
                                                      'cdom_score','bellwether'])
        score_df = score_df.sort_values('id')
        score_df.to_csv(data_source1 + '/bellwether_cdom_2.csv')
        level_1_bellwethers = {}
        for cluster in cluster_structure[2].keys():
            if cluster_structure[2][cluster] not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster_structure[2][cluster]] = []
            level_1_bellwethers[cluster_structure[2][cluster]].append(score_df[score_df['id'] == cluster].bellwether.values[0])
        for cluster in cluster_structure[1].keys():
            if cluster not in level_1_bellwethers.keys():
                level_1_bellwethers[cluster] = []
                level_1_bellwethers[cluster] = list(df_train.iloc[cluster_tree[cluster].data_points].index)
        score_med = []
        print(level_1_bellwethers.keys())
        for key in  level_1_bellwethers.keys():
            sub_cluster_bellwethers = level_1_bellwethers[key]
            print(key,len(sub_cluster_bellwethers))
            bell = birch_bellwether.bellwether(path,df_train)
            final_score = bell.bellwether(sub_cluster_bellwethers,sub_cluster_bellwethers)
            with open(data_source1 + '/cdom_level1/cluster_'  + str(key) + '_performance.pkl', 'wb') as handle:
                pickle.dump(final_score, handle, protocol=pickle.HIGHEST_PROTOCOL)             

In [6]:
for i in range(20):
    start = timeit.default_timer()
    print("this is iteration =====", i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/2/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,clusters,path,i)
    stop = timeit.default_timer() 
    print("Model training time: ", stop - start)

this is iteration ===== 0
{0: {0: None}, 1: {1: 0, 2: 0, 3: 0, 9: 0, 20: 0, 37: 0, 38: 0}, 2: {4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 10: 9, 11: 9, 12: 9, 13: 9, 14: 9, 15: 9, 16: 9, 17: 9, 18: 9, 19: 9, 21: 20, 22: 20, 23: 20, 24: 20, 25: 20, 26: 20, 27: 20, 28: 20, 29: 20, 30: 20, 31: 20, 32: 20, 33: 20, 34: 20, 35: 20, 36: 20, 39: 38, 40: 38, 41: 38, 42: 38, 43: 38, 44: 38, 45: 38, 46: 38, 47: 38, 48: 38, 49: 38, 50: 38, 51: 38, 52: 38, 53: 38, 54: 38, 55: 38, 56: 38, 57: 38, 58: 38}}
dict_keys([3, 9, 20, 38, 1, 2, 37])
3 5
ossim.csv
les-indemodables.csv
amateur-scrolls.csv
vars-redux.csv
foursquared.csv
9 10
powerfolder-.csv
planeshift.csv
maidsafe-dht.csv
ardour.csv
google-caja.csv
autowikibrowser.csv
wxcrp.csv
libarchive.csv
ftm.csv
netatalk.csv
20 16
mule.csv
amygdala.csv
jpaul.csv
mclient-mume.csv
opennms.csv
tango-cs.csv
voms.csv
red5.csv
bionote.csv
kolmafia.csv
zildo.csv
wxcode.csv
lite.csv
igarden.csv
panda3d.csv
irplus.csv
38 20
jbasic.csv
checker-framework.csv
mptvseries.csv
anaxa

ktc.csv
acdk.csv
novembre.csv
mcmc-jags.csv
zildo.csv
k3d.csv
rocrail.csv
aime.csv
mptvseries.csv
xbplayer.csv
fulguro.csv
qgo.csv
flylinkdc.csv
tolven.csv
autat.csv
netatalk.csv
52 14
olex2.csv
jcl.csv
alembik.csv
htmlunit.csv
vxl.csv
konversation.csv
intact.csv
wxcode.csv
orc.csv
opennms.csv
pdfedit.csv
pclasses.csv
tradelink.csv
jaxen.csv
24 2
javagroups.csv
magicwars.csv
25 3
esmf.csv
pentahoanalysistool.csv
turbotrader-bos.csv
26 1
xmemcached.csv
Model training time:  24.00233694000002
this is iteration ===== 6
{0: {0: None}, 1: {1: 0, 9: 0, 12: 0, 23: 0, 24: 0, 25: 0, 26: 0, 32: 0, 51: 0}, 2: {2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 10: 9, 11: 9, 13: 12, 14: 12, 15: 12, 16: 12, 17: 12, 18: 12, 19: 12, 20: 12, 21: 12, 22: 12, 27: 26, 28: 26, 29: 26, 30: 26, 31: 26, 33: 32, 34: 32, 35: 32, 36: 32, 37: 32, 38: 32, 39: 32, 40: 32, 41: 32, 42: 32, 43: 32, 44: 32, 45: 32, 46: 32, 47: 32, 48: 32, 49: 32, 50: 32, 52: 51, 53: 51, 54: 51, 55: 51, 56: 51, 57: 51, 58: 51, 59: 51, 60: 51, 6

dict_keys([2, 23, 31, 36, 49, 1])
2 20
encog-java.csv
fontforge.csv
mcmc-jags.csv
amateur-scrolls.csv
openmeetings.csv
httpcontentparser.csv
alsa.csv
konversation.csv
sigil.csv
exordium.csv
motu.csv
playerstage.csv
haphazard.csv
turbotrader-bos.csv
libarchive.csv
gzigzag.csv
unimrcp.csv
dataobjectsdotnet.csv
skia.csv
neoengine.csv
23 7
jmoney.csv
m-m-m.csv
coot.csv
archive-crawler.csv
jrdf.csv
ibatisdb.csv
geotools.csv
31 4
bloodycore.csv
v8.csv
mumble.csv
novembre.csv
36 12
lamp.csv
guineu.csv
owlapi.csv
javagroups.csv
jam-daq.csv
mevenide.csv
pustefix.csv
owfs.csv
crisscross.csv
tokratan.csv
botsnscouts.csv
snakeyaml.csv
49 14
cuberok.csv
gbif-providertoolkit.csv
tango-cs.csv
thrust.csv
wgs-assembler.csv
rocrail.csv
xbplayer.csv
jahshakafx.csv
opennms.csv
jgameai.csv
nebuladevice.csv
projecteqemu.csv
img-browse-sort.csv
quark.csv
1 1
atunes.csv
Model training time:  25.06879030799996
this is iteration ===== 12
{0: {0: None}, 1: {1: 0, 11: 0, 12: 0, 15: 0, 29: 0, 50: 0, 56: 0, 60: 0},

fulguro.csv
lite.csv
bulmages.csv
emisview.csv
wxcode.csv
tzod.csv
nassp.csv
jrdf.csv
rodin-b-sharp.csv
23 2
javagroups.csv
magicwars.csv
24 5
esmf.csv
pentahoanalysistool.csv
bigdata.csv
wsmt.csv
codesmith.csv
25 1
xmemcached.csv
Model training time:  26.61837052200002
this is iteration ===== 17
{0: {0: None}, 1: {1: 0, 9: 0, 12: 0, 23: 0, 24: 0, 25: 0, 26: 0, 31: 0, 51: 0}, 2: {2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 10: 9, 11: 9, 13: 12, 14: 12, 15: 12, 16: 12, 17: 12, 18: 12, 19: 12, 20: 12, 21: 12, 22: 12, 27: 26, 28: 26, 29: 26, 30: 26, 32: 31, 33: 31, 34: 31, 35: 31, 36: 31, 37: 31, 38: 31, 39: 31, 40: 31, 41: 31, 42: 31, 43: 31, 44: 31, 45: 31, 46: 31, 47: 31, 48: 31, 49: 31, 50: 31, 52: 51, 53: 51, 54: 51, 55: 51, 56: 51, 57: 51, 58: 51, 59: 51, 60: 51, 61: 51, 62: 51, 63: 51, 64: 51, 65: 51, 66: 51, 67: 51, 68: 51, 69: 51}}
dict_keys([1, 9, 12, 26, 31, 51, 23, 24, 25])
1 7
mclient-mume.csv
jahshaka.csv
icescrum.csv
nunit.csv
azureus.csv
sblim.csv
synecdoche.csv
9 2
atunes.c